In [ ]:
import numpy as np

mu = 0.5
sigma = 1

t = 0.45

def est_deriv_mu(samples):
    approx = np.square(samples * sigma + mu - t) * samples / sigma
    return np.mean(approx)

def est_deriv_sigma(samples):
    approx = np.square(samples * sigma + mu - t) * (np.square(samples) - 1) / sigma
    return np.mean(approx)

def get_noise(num):
        return np.random.normal(size=(num, 1))


for i in range(7):
    num_samples = 10 ** i
    samples = get_noise(num_samples)
    est_mu, est_sigma = est_deriv_mu(samples), est_deriv_sigma(samples)
    print(est_mu, est_sigma)